# Homework Assignment 2: QR Factorization

One algorithmic idea in numerical linear algebra is more important ([one of the 10 most important algorithms of the 20th century](https://archive.siam.org/pdf/news/637.pdf)) than all the others: QR factorization. This decomposition plays a crucial role in many problems:
* Computing orthogonal bases in a linear space.
* The preprocessing step for the singular value decomposition (principal component analysis).
* The computation of eigenvectors and eigenvalues (covered in next week's lectures).
* Methods for least squares problems (overdetermined systems of linear equations).

Before tackling the task of implementing this method, let's first introduce the Python library that provides all the necessary linear algebra functionality.

## The NumPy Library

NumPy is the fundamental package for scientific computing with Python, adding support for large, multidimensional arrays and matrices, along with a large collection of mathematical functions to operate on these arrays. It is the workhorse library for linear algebra. To use the NumPy library we need to import it first.

In [1]:
# Importing the NumPy module and giving it the conventional shorthand name np.
import numpy as np 

### Creating arrays from Python lists

To create a vector, simply surround a python list with the `array` function:

In [2]:
np.array([1, 4, 2, 5, 3])

array([1, 4, 2, 5, 3])

**Exercise 1:** Create an array of the numbers from 0 to 20.

In [3]:
np.array(range(0,21))

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20])

Unlike Python lists, NumPy is constrained to arrays that all contain the same datatype. If types do not match, NumPy will upcast if possible (here, integers are up-cast to floating points):

In [4]:
np.array([3.14, 4, 2, 3])

array([3.14, 4.  , 2.  , 3.  ])

If we want to explicitly set the datatype of the resulting array, we can use the `dtype` argument:

In [5]:
np.array([1, 2, 3, 4], dtype='float64')

array([1., 2., 3., 4.])

Finally, unlike Python lists, NumPy arrays can explicitly be multidimensional; here is one way of initializing a multidimensional array using a list of lists. The inner lists are treated as rows of the resulting two-dimensional array.

In [6]:
arr = np.array([[2, 3, 4], 
                [4, 5, 6], 
                [6, 7, 8]])
print(arr)

[[2 3 4]
 [4 5 6]
 [6 7 8]]


### Creating arrays from scratch

Especially for larger arrays, it is more efficient to create arrays from scratch using routines built into NumPy. Here are several examples:

In [7]:
# Create a length-10 integer array filled with zeroes
arr1 = np.zeros(10, dtype=int)

# Create a 3x5 floating-point array filled with ones
arr2 = np.ones((3, 5), dtype=float)

# Create a 3x5 array filled with 3.14
arr3 = np.full((3, 5), 3.14)

# Create an array filled with a linear sequence
# starting at 0, ending at 20, steps by 2 (similar to range)
arr4 = np.arange(0, 20, 2)

# Create an array of five values evenly spaces between 0 and 1
arr5 = np.linspace(0, 1, 5)

# Create a 3x3 array of uniformly ditributed random values between 0 and 1
arr6 = np.random.random((3, 3))

# Create a 3x3 identity matrix
arr7 = np.eye(3)


print("np.zeros", "\n", arr1, "\n")
print("np.ones", "\n", arr2, "\n")
print("np.full", "\n", arr3, "\n")
print("np.arange", "\n", arr4, "\n")
print("np.linspace", "\n", arr5, "\n")
print("np.random.random", "\n", arr6, "\n")
print("np.eye", "\n", arr7)

np.zeros 
 [0 0 0 0 0 0 0 0 0 0] 

np.ones 
 [[1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]] 

np.full 
 [[3.14 3.14 3.14 3.14 3.14]
 [3.14 3.14 3.14 3.14 3.14]
 [3.14 3.14 3.14 3.14 3.14]] 

np.arange 
 [ 0  2  4  6  8 10 12 14 16 18] 

np.linspace 
 [0.   0.25 0.5  0.75 1.  ] 

np.random.random 
 [[0.67082294 0.88225711 0.57599585]
 [0.36110937 0.49829054 0.34822523]
 [0.57976964 0.63983841 0.26352591]] 

np.eye 
 [[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


Each array has attributes `ndim` (the number of dimensions), `shape` (the size of each dimension), `size` (the total size of the array), and `dtype` (the datatype of the array):

In [8]:
print("arr3 ndim:", arr3.ndim)
print("arr3 shape:", arr3.shape)
print("arr3 size:", arr3.size)
print("arr3 dtype:", arr3.dtype)

arr3 ndim: 2
arr3 shape: (3, 5)
arr3 size: 15
arr3 dtype: float64


**Exercise 2:** In contrast to Python's list objects, arithmetic operations on a ndarray are executed element-by-element. Use this to create and print the following ndarrays:
* element-by-element sum of `arr` and `arr6`.
* element-by-element product of `arr` and `arr6`.
* elements of `arr` multiplied by a factor of 10.
* elements of `arr` to the power 4.
* one divided by the elements of `arr`.

In [9]:
print(arr + arr6)
print(arr * arr6)
print(arr * 20)
print(arr ** 4)
print(1 / arr)

[[2.67082294 3.88225711 4.57599585]
 [4.36110937 5.49829054 6.34822523]
 [6.57976964 7.63983841 8.26352591]]
[[1.34164587 2.64677133 2.30398338]
 [1.44443748 2.49145271 2.0893514 ]
 [3.47861786 4.47886889 2.10820726]]
[[ 40  60  80]
 [ 80 100 120]
 [120 140 160]]
[[  16   81  256]
 [ 256  625 1296]
 [1296 2401 4096]]
[[0.5        0.33333333 0.25      ]
 [0.25       0.2        0.16666667]
 [0.16666667 0.14285714 0.125     ]]


### Array indexing and slicing

In a one-dimensional array, the $i^{\text{th}}$ value (counting from zero) can be accessed by specifying the desired index in square brackets, just as with Python lists. Negative indices can be used for indexing from the end of the array. In multidimensional arrays, items can be accessed using a comma-separated tuple of indices. 

In [10]:
print(arr4)
print("Fourth element:", arr4[3], "\n")
print(arr4)
print("Second to last element:", arr4[-2], "\n")
print(arr4)
print("Sub-array of fourth and seventh element:", arr4[[3, 6]], "\n")
print(arr)
print("Element in first row, second column:", arr[0,1])

[ 0  2  4  6  8 10 12 14 16 18]
Fourth element: 6 

[ 0  2  4  6  8 10 12 14 16 18]
Second to last element: 16 

[ 0  2  4  6  8 10 12 14 16 18]
Sub-array of fourth and seventh element: [ 6 12] 

[[2 3 4]
 [4 5 6]
 [6 7 8]]
Element in first row, second column: 3


Just as we can use square brackets to access individual array elements, we can also use them to access subarrays with the *slice* notation, marked by the colon `:` character. The NumPy slicing syntax follows that of standard Python lists: `array[start:stop:step]`. 

In [11]:
print(arr4)

print("First five elements:", arr4[:5])

print("Elements after index 5:", arr4[5:])

print("Middle subarray:", arr4[4:7])

print("Every other element:", arr4[::2])

print("Every other element starting at index 1:", arr4[1::2])

[ 0  2  4  6  8 10 12 14 16 18]
First five elements: [0 2 4 6 8]
Elements after index 5: [10 12 14 16 18]
Middle subarray: [ 8 10 12]
Every other element: [ 0  4  8 12 16]
Every other element starting at index 1: [ 2  6 10 14 18]


**Note**: When we treat the array as a formal matrix (on which we would like to perform linear algebraic operations like matrix-vector and matrix-matrix multiplication) we have to take care when considering submatrices. Do note that the difference between the slices `A[:,0]` and `A[:,0:1]`, where the first is turned into a one dimensional array, whereas the second is a correct column vector (a two-dimensional array). The latter is required for matrix-vector multiplications. 

In [12]:
print("Original array\n", arr, "\n")
print("Subarray using arr[:,0]\n", arr[:,0])
print("Dimension subarray =", arr[:,0].ndim, "\n")
print("Subarray using arr[:,0:1]\n", arr[:,0:1])
print("Dimension subarray =", arr[:,0:1].ndim)

Original array
 [[2 3 4]
 [4 5 6]
 [6 7 8]] 

Subarray using arr[:,0]
 [2 4 6]
Dimension subarray = 1 

Subarray using arr[:,0:1]
 [[2]
 [4]
 [6]]
Dimension subarray = 2


In [13]:
# Arrays for exercise 3 (see below)
arr1D = np.arange(1,11)
arr2D = np.arange(1,25).reshape(4,6)
arr3D = np.arange(1,13).reshape(2,2,3)

print('arr1D:\n {0}'.format(arr1D))
print('arr2D:\n {0}'.format(arr2D))
print('arr3D:\n {0}'.format(arr3D))

arr1D:
 [ 1  2  3  4  5  6  7  8  9 10]
arr2D:
 [[ 1  2  3  4  5  6]
 [ 7  8  9 10 11 12]
 [13 14 15 16 17 18]
 [19 20 21 22 23 24]]
arr3D:
 [[[ 1  2  3]
  [ 4  5  6]]

 [[ 7  8  9]
  [10 11 12]]]


**Exercise 3:** use the arrays `arr1D`, `arr2D` and `arr3D` defined above to print the following objects.
* for `arr1D`: the fifth element.
* for `arr1D`: the array containing (only) the fifth element.
* for `arr1D`: the array containing the third until the nineth element.
* for `arr1D`: the array containing the first, second and seventh element.


* for `arr2D`: the element in the second row and third column.
* for `arr2D`: the array that is the fourth row.
* for `arr2D`: the array containing the second and the third row.
* for `arr2D`: the array containing the first and third row.
* for `arr2D`: the array containing the first and second row, but only the third until the fifth column thereof.
* for `arr2D`: the array containing the fifth and sixth column.
* for `arr2D`: the array containing the fifth and sixth column, which are represented as rows. (Hint: use `tranpose()`, or simply `T`)


* for `arr3D`: the first 2-dimensional array of `arr3D`.
* for `arr3D`: the 2-dimensional array that looks like `[[5,6],[11,12]]`.

## Problem 1: Getting started with arrays (2 points)

Implement the two functions `pascal` and `flip` below, see instructions in the corresponding docstrings. 

In [14]:
def pascal(n):
    """
    Returns a two dimensional n by n array P that contains the binomial coefficient (i choose j)
    at entry (i,j). Recall that the entries in Pascal's triangle are the sum of the two entries 
    above it, see for reference https://en.wikipedia.org/wiki/Pascal%27s_triangle
    
    Example
    ----------
    pascal(5) =
        [[1 0 0 0 0]
         [1 1 0 0 0]
         [1 2 1 0 0]
         [1 3 3 1 0]
         [1 4 6 4 1]]
    
    Parameters
    ----------
        n (int): Order of the n by n array
        
    Returns
    -------
        P (ndarray): Array filled with a Pascal triangle pattern
    
    """
    
    # YOUR CODE HERE
    P = np.zeros(n ** 2, dtype = int).reshape(n, n)
    P[0, 0] = 1

    if n == 1:
        return P

    else:
        for i in range(1, n):
            for j in range(0, n):
                P[i, j] = P[i-1, j] + P[i-1, j-1]

        return(P)

In [15]:
def flip(arr):
    """
    Returns the left-right flipped array arr,
    where the entries in each row are flipped in the 
    left/right direction.
    
    Example
    -------
    arr = [[1 2 3]
           [4 5 6]
           [7 8 9]]
           
    flip(arr) = 
        [[3 2 1]
         [6 5 4]
         [9 8 7]]
    
    Parameters
    ----------
        arr (ndarray): Any arbitrary two-dimensional array
        
    Returns
    -------
        arr_flip (ndarray): Two-dimensional array arr flipped left/right

    """
    
    # YOUR CODE HERE
    nrow, ncol = arr.shape
    arr_flip = np.zeros(nrow * ncol).reshape(nrow, ncol)

    for i in range(0, nrow):
        for j in range(0, ncol):
            arr_flip[i, j-1] = arr[i, -j]

    return(arr_flip)

In [16]:
# You can use this code cell to play around with your functions to make sure
# they do what they are intended to do, i.e. to debug your code. 


In [17]:
# Test case
P = pascal(5)
print(P, "\n")

arr = np.array([[1, 2, 3, 4, 5], 
                [6, 7, 8, 9, 10], 
                [11, 12, 13, 14, 15]])
print(flip(arr), "\n")

[[1 0 0 0 0]
 [1 1 0 0 0]
 [1 2 1 0 0]
 [1 3 3 1 0]
 [1 4 6 4 1]] 

[[ 5.  4.  3.  2.  1.]
 [10.  9.  8.  7.  6.]
 [15. 14. 13. 12. 11.]] 



Expected output:

    [[1 0 0 0 0]
     [1 1 0 0 0]
     [1 2 1 0 0]
     [1 3 3 1 0]
     [1 4 6 4 1]] 

    [[ 5  4  3  2  1]
     [10  9  8  7  6]
     [15 14 13 12 11]]

In [18]:
# AUTOGRADING for pascal
P = np.array([[1, 0, 0, 0, 0],
              [1, 1, 0, 0, 0],
              [1, 2, 1, 0, 0],
              [1, 3, 3, 1, 0],
              [1, 4, 6, 4, 1]])
assert np.allclose(P, pascal(5))


In [19]:
# AUTOGRADING for flip
arr = np.array([[1, 2, 3, 4, 5], 
                [6, 7, 8, 9, 10], 
                [11, 12, 13, 14, 15]])
F = np.array([[5, 4, 3, 2, 1],
              [10, 9, 8, 7, 6],
              [15, 14, 13, 12, 11]])
assert np.allclose(F, flip(arr))


## Standard Gram-Schmidt Process

Recall that every rectangular $n \times m$ ($n \geq m$) matrix $A$ with linearly indepedent columns can be represented as a product
$$A = QR, \qquad \text{or in matrix form} \qquad \begin{bmatrix}\mid & \mid & & \mid \\ v_1 & v_2 & \dots & v_m \\ \mid & \mid & & \mid \end{bmatrix} = \begin{bmatrix}\mid & \mid & & \mid \\ u_1 & u_2 & \dots & u_m \\ \mid & \mid & & \mid \end{bmatrix} \begin{bmatrix} r_{11} & r_{12} & \dots & r_{1m} \\ & r_{22} & \dots & r_{2m}\\ & & \ddots & \vdots \\ & & & r_{mm} \end{bmatrix}$$
where $Q$ is an $n \times m$ matrix whose columns are orthonormal and $R$ is an upper-triangular $m \times m$ matrix. In this assignment your task will be to implement several methods to achieve this factorization for a given matrix $A$. 

Our first goal is to orthogonalize a system of linearly independent vectors $v_1, \dots, v_m$ (i.e. the columns of some matrix $A$). The standard procedure for this task is the Gram-Schmidt process as covered in the lecture:
\begin{align*}
u_1 &= \frac{v_1}{r_{11}} \\
v_2^\perp &= v_2 - r_{12} u_1, \quad u_2 = \frac{v_2^\perp}{r_{22}} \\
v_3^\perp &= v_3 - r_{13} u_1 - r_{23} u_2, \quad u_3 = \frac{v_2^\perp}{r_{33}} \\
\vdots
\end{align*}
where
\begin{align*}
r_{jj} &= \|v_j - \sum_{i=1}^{j-1} r_{ij} u_i\| &&\quad \text{for }\ 1 \leq j \leq m\ \\
r_{ij} &= \langle v_j, u_i \rangle &&\quad \text{for } 1 \leq i < j \leq m
\end{align*}

## Problem 2: (Classical) Gram-Schmidt Orthogonalization (3 points)

Implement the described Gram-Schmidt algorithm as the function `gram_schmidt_qr` below that takes a rectangular array $A$ and outputs $Q$ and $R$. 

**IMPORTANT**: Do not use any built-in QR functions like `np.linalg.qr` or `scipy.lingalg.qr`. These will give different results anyway, as these use a different numerical scheme. 

**HINT**: The following functions for vectors might be useful: `np.linalg.norm` to compute the norm of a 1-D array and `np.inner` to compute the regular dot product between two 1-D arrays.

In [20]:
def gram_schmidt_qr(A):
    """
    Computes the QR factorization of a matrix A with linearly independent columns
    by means of the classical Gram-Schmidt process, where Q is an orthogonal matrix 
    and R is upper-triangular with positive entries on its diagonal. 
    
    Parameters
    ----------
        A (ndarray): A two-dimensional array of size n by m with n >= m
            whose columns are linearly independent
    
    Returns
    -------
        Q (ndarray): A two-dimensional array of size n by m
        R (ndarray): A two-dimensional array of size m by m
        
    """
    
    n, m = A.shape
    Q = np.zeros((n,m))
    R = np.zeros((m,m))
    
    # YOUR CODE HERE
    R[0,0] = np.linalg.norm(A[:,0])
    Q[:,0] = A[:,0] / R[0,0]

    for j in range(1, m):
        v = A[:,j]
        for i in range(0, j):
            R[i, j] = np.inner(Q[:,i], A[:,j])
            v = v - R[i, j] * Q[:,i]
        R[j, j] = np.linalg.norm(v)
        Q[:,j] = v / R[j, j]
    
    return Q, R


In [21]:
# You can use this code cell to play around with your function to make sure
# it does what it is intended to do, i.e. to debug your code. 

In [22]:
# Test case I
np.set_printoptions(suppress=True) # Prints nicer numbers, you may ignore this

A = np.array([[2,2], [1,7], [-2, -8]])
Q, R = gram_schmidt_qr(A)
print(Q, "\n")
print(R)

[[ 0.66666667 -0.66666667]
 [ 0.33333333  0.66666667]
 [-0.66666667 -0.33333333]] 

[[3. 9.]
 [0. 6.]]


Expected output:

    [[ 0.66666667 -0.66666667]
     [ 0.33333333  0.66666667]
     [-0.66666667 -0.33333333]]
     
    [[3. 9.]
     [0. 6.]]

In [23]:
# Test case II
B = np.array([[-1, 4, -1], [1, -1, -1], [1, -1, 1], [-1, 4, -3]])
Q, R = gram_schmidt_qr(B)
print(Q, "\n")
print(R)

[[-0.5  0.5  0.5]
 [ 0.5  0.5 -0.5]
 [ 0.5  0.5  0.5]
 [-0.5  0.5 -0.5]] 

[[ 2. -5.  2.]
 [ 0.  3. -2.]
 [ 0.  0.  2.]]


Expected output:

    [[-0.5  0.5  0.5]
     [ 0.5  0.5 -0.5]
     [ 0.5  0.5  0.5]
     [-0.5  0.5 -0.5]]
     
    [[ 2. -5.  2.]
     [ 0.  3. -2.]
     [ 0.  0.  2.]]

In [24]:
# AUTOGRADER
A = np.array([[2,2], [1,7], [-2, -8]])
Q, R = gram_schmidt_qr(A)
Q_exact, R_exact = 1/3*np.array([[2, -2], [1, 2], [-2, -1]]), np.array([[3, 9], [0, 6]])

assert np.allclose(Q, Q_exact)
assert np.allclose(R, R_exact)

B = np.array([[-1, 4, -1], [1, -1, -1], [1, -1, 1], [-1, 4, -3]])
Q, R = gram_schmidt_qr(B)
Q_exact, R_exact = 1/2 * np.array([[-1, 1, 1], [1, 1, -1], [1, 1, 1], [-1, 1, -1]]), np.array([[2, -5, 2], [0, 3, -2], [0, 0, 2]])

assert np.allclose(Q, Q_exact)
assert np.allclose(R, R_exact)


## Givens Triangularization 

Another principal method for computing QR factorizations is Givens triangularization, which is numerically more stable than Gram-Schmidt orthogonalization. The Givens algorithm is a process of "orthogonal triangularization", making a matrix triangular by a sequence of orthogonal matrix operations. 

In contrast to the Gram-Schmidt iteration which applies a succession of elementary triangular matrices $R_i$ on the right of $A$ resulting in an orthogonal matrix (see optional section at the end of this assignment), the Givens method applies a succession of elementary orthogonal matrices $Q_i$ on the left of $A$ so that the resulting matrix
$$Q_k \dots Q_2 Q_1 A = R$$
is upper-triangular. The product $Q = Q_1^T Q_2^T \dots Q_k^T$ is also orthogonal, and therefore $A = QR$ is our $QR$ factorization of $A$. We can summarize this method as orthogonal triangularization. 

### Givens Rotations

At the heart of the Givens method are orthogonal matrices to introduce zeroes. To illustrate how we create zeroes we consider the case where $A$ is a $2 \times 2$ matrix, for which we only need to make $a_{21}$ zero. The QR factorization computes $Q A = R$, or
$$\begin{bmatrix}
c & -s \\
s & c \end{bmatrix} \begin{bmatrix} a_{11} & a_{12} \\ a_{21} & a_{22} \end{bmatrix} 
= \begin{bmatrix}
r_{11} & r_{12} \\
0 & r_{22} \end{bmatrix},$$
where $c^2 + s^2 = 1$ to ensure $Q$ is orthogonal. We have $c = \cos(\theta)$ and $s = \sin(\theta)$ for some $\theta$. 

From the relationship $s a_{11} + c a_{21} = 0$ we obtain
$$c^2 a_{21}^2 = s^2 a_{11}^2 = (1-c^2) a_{11}^2,$$
which yields
$$c = \pm \frac{a_{11}}{\sqrt{a_{21}^2 + a_{11}^2}}.$$

For now we choose the $+$ sign. Then we obtain
$$s^2 = 1-c^2 = 1 - \frac{a_{11}^2}{a_{21}^2 + a_{11}^2} = \frac{a_{21}^2}{a_{21}^2 + a_{11}^2},$$
or
$$s = \pm \frac{a_{21}}{\sqrt{a_{21}^2 + a_{11}^2}}.$$

To ensure that $sa_{11} + ca_{21}=0$ we choose the $-$ sign. As a result we have
$$r_{11} = a_{11} \frac{a_{11}}{\sqrt{a_{21}^2 + a_{11}^2}} - a_{21} \frac{-a_{21}}{\sqrt{a_{21}^2 + a_{11}^2}} = \sqrt{a_{21}^2 + a_{11}^2}.$$

The matrix $Q = \begin{bmatrix} c & -s \\ s & c \end{bmatrix}$ is called a Givens rotation. It is called a rotation because it is orthogonal, and therefore length preserving, and also because there is an angle $\theta$ such that $\sin(\theta) = s$ and $\cos(\theta) = c$. 

The effect of pre-multiplying a vector by $Q$ is to rotate the vector counterclockwise through the angle $\theta$. In particular, if $a = r\cos(-\theta)$ and $b = r\sin(-\theta)$, where the negative sign in front of the angle $\theta$ is for illustrative purposes, then
$$\begin{bmatrix} c & -s \\ s & c \end{bmatrix} \begin{bmatrix} a \\ b \end{bmatrix} = \begin{bmatrix} r \\ 0 \end{bmatrix}, \quad r = \sqrt{a^2 + b^2}.$$
That is, the point $(a,b)$ is rotated to the positive $x$-axis, effectively "undoing" the counterclockwise rotation by $-\theta$ inherent in polar coordinates. 

Now to see how Givens rotations can be used to create zero entries of an $n \times m$ matrix $A$, suppose that we have the vector
$$\begin{bmatrix} \times \\ \vdots \\ \times \\ a \\ \times \\ \vdots \\ \times \\ b \\ \times \\ \vdots \\ \times \end{bmatrix}$$
that is a column of $A$. Then
$$\begin{bmatrix} 1 & & & & & & & & & & \\
& \ddots & & & & & & & & & \\
& & 1 & & & & & & & & \\
& & & c & & & & -s & & & \\
& & & & 1 & & & & & & \\
& & & & & \ddots & & & & & \\
& & & & & & 1 & & & & \\
& & & s & & & & c & & & \\
& & & & & & & & 1 & & \\
& & & & & & & & & \ddots & \\
& & & & & & & & & & 1\\
\end{bmatrix} \begin{bmatrix} \times \\ \vdots \\ \times \\ a \\ \times \\ \vdots \\ \times \\ b \\ \times \\ \vdots \\ \times \end{bmatrix} = \begin{bmatrix} \times \\ \vdots \\ \times \\ \pmb{r} \\ \times \\ \vdots \\ \times \\ \pmb{0} \\ \times \\ \vdots \\ \times \end{bmatrix},$$
where blank entries are zero.

### Successive rotations

To transform $A$ into an upper triangular matrix $R$, we can find a product of rotations $Q$ such that $Q A = R$ as illustrated below. In these matrices, the symbol $\times$ represents an entry that is not necesssarily zero, and boldfacing them indicates an entry that has just been changed.  

\begin{align*}
\begin{bmatrix}
\times & \times & \times \\ 
\times & \times & \times \\ 
\times & \times & \times \\ 
\times & \times & \times \\ 
\times & \times & \times 
\end{bmatrix} &\xrightarrow{Q_1} \begin{bmatrix}
\times & \times & \times \\ 
\times & \times & \times \\ 
\times & \times & \times \\ 
\pmb{\times} & \pmb{\times} & \pmb{\times} \\ 
\pmb{0} & \pmb{\times} & \pmb{\times} 
\end{bmatrix} \xrightarrow{Q_2} \begin{bmatrix}
\times & \times & \times \\ 
\times & \times & \times \\ 
\pmb{\times} & \pmb{\times} & \pmb{\times} \\ 
\pmb{0} & \pmb{\times} & \pmb{\times} \\ 
0 & \times & \times
\end{bmatrix} \xrightarrow{Q_3} \begin{bmatrix}
\times & \times & \times \\ 
\pmb{\times} & \pmb{\times} & \pmb{\times} \\ 
\pmb{0} & \pmb{\times} & \pmb{\times} \\ 
0 & \times & \times \\
0 & \times & \times 
\end{bmatrix} \xrightarrow{Q_4} \begin{bmatrix}
\pmb{\times} & \pmb{\times} & \pmb{\times} \\ 
\pmb{0} & \pmb{\times} & \pmb{\times} \\ 
0 & \times & \times \\
0 & \times & \times \\
0 & \times & \times 
\end{bmatrix} \\
&\xrightarrow{Q_5} \begin{bmatrix}
\times & \times & \times \\ 
0 & \times & \times \\ 
0 & \times & \times \\
0 & \pmb{\times} & \pmb{\times} \\
0 & \pmb{0} & \pmb{\times} 
\end{bmatrix} \xrightarrow{Q_6} \begin{bmatrix}
\times & \times & \times \\ 
0 & \times & \times \\
0 & \pmb{\times} & \pmb{\times} \\
0 & \pmb{0} & \pmb{\times} \\
0 & 0 & \times 
\end{bmatrix} \xrightarrow{Q_7} \begin{bmatrix}
\times & \times & \times \\ 
0 & \pmb{\times} & \pmb{\times} \\ 
0 & \pmb{0} & \pmb{\times} \\
0 & 0 & \times \\
0 & 0 & \times 
\end{bmatrix} \\
& \xrightarrow{Q_8} \begin{bmatrix}
\times & \times & \times \\ 
0 & \times & \times \\ 
0 & 0 & \times \\
0 & 0 & \pmb{\times} \\
0 & 0 & \pmb{0} 
\end{bmatrix} \xrightarrow{Q_9} \begin{bmatrix}
\times & \times & \times \\ 
0 & \times & \times \\ 
0 & 0 & \pmb{\times} \\
0 & 0 & \pmb{0} \\
0 & 0 & 0
\end{bmatrix}
\end{align*}


### Overflow and underflow issues

It is important to note that the straightforward approach to computing the entries $c$ and $s$ of the Givens rotation,
$$c = \frac{a}{\sqrt{a^2 + b^2}}, \quad s = \frac{-b}{\sqrt{a^2+b^2}}$$
is not always advisable, because in floating-point arithmetic, the computation of $\sqrt{a^2 + b^2}$ could overflow or underflow. To get around this problem, suppose that $|a| \geq |b|$. Then, we can instead compute
$$t = \frac{b}{a}, \quad c = \frac{\text{sgn}(a)}{\sqrt{1+t^2}}, \quad s = -c t,$$
which is guaranteed not to overflow since the only number that is squared is at most one in magnitude.

Similarly, if $|b| \geq |a|$, then we compute
$$t = \frac{a}{b}, \quad s = \frac{-\text{sgn}(b)}{\sqrt{1+t^2}}, \quad c = -st.$$

In case $b=0$ we can take
$$c = \text{sgn}(a), \quad s = 0,$$
and in case $a=0$ we can take
$$c = 0, \quad s = -\text{sgn}(b).$$

Here $\text{sgn}(x)$ represent the sign of x, that is, $\text{sgn}(x) = 1$ if $x \geq 0$ and $\text{sgn}(x) = -1$ if $x<0$.

**Note**: In this QR factorization the rectangular $n \times m$ ($n \geq m$) matrix $A$ with linearly indepedent columns will be represented as a product
$$A = QR, \qquad \text{or in matrix form} \qquad \begin{bmatrix}\mid & \mid & & \mid \\ v_1 & v_2 & \dots & v_m \\ \mid & \mid & & \mid \end{bmatrix} = \begin{bmatrix}\mid & \mid & & \mid \\ u_1 & u_2 & \dots & u_n \\ \mid & \mid & & \mid \end{bmatrix} \begin{bmatrix} r_{11} & r_{12} & \dots & r_{1m} \\ & r_{22} & \dots & r_{2m}\\ & & \ddots & \vdots \\ & & & r_{mm} \\ & & & \end{bmatrix}$$
where $Q$ is an orthogonal $n \times n$ matrix and $R$ is an upper-triangular $n \times m$ matrix. So when $A$ is not square, $R$ will contain extra rows of zeroes at the bottom. Moreover, we also allow for negative entries along the diagonal (as opposed to the QR factorization we obtain by applying the Gram-Schmidt process). 

## Problem 3: Givens QR Factorization (5 = 2 + 3 points)

Implement both the functions `givens_rotation` and `givens_qr` below. 

**Hint:** You can multiply two 2 dimensional arrays (i.e. matrices) $A$ and $B$ by `A @ B` or by `np.matmul(A,B)`. You can compute the inverse of a matrix by [`np.lingalg.inv`](https://numpy.org/doc/stable/reference/generated/numpy.linalg.inv.html) and the transpose of a matrix by [`np.transpose`](https://numpy.org/doc/stable/reference/generated/numpy.transpose.html).

In [25]:
def sign(x):
    """
    To be used in givens_rotation below in the computation of
    the absolute value
    (The builtin function np.sign gives zero the sign zero, 
    which we don't want)
    """
    if(x >= 0):
        return(1)
    else:
        return(-1)

In [26]:
def givens_rotation(i, j, a, b, n):
    """
    Returns the Givens rotation matrix Q that rotates a vector x
    of length n with x[i] = a and x[j] = b to a vector y=Qx 
    with y[i] = r >= 0 and y[j] = 0.
    
    Parameters
    ----------
        i (int): integer between 0 and (n-1) representing i^th entry of vector x
        j (int): integer between i and (n-1) representing j^th entry of x
        a (float): i^th coefficient of vector x
        b (float): j^th coefficient of vector x
        n (int): length of vector x
        
    Returns
    -------
        Q (ndarray): A two-dimensional array of size n by n
    
    """
    
    # YOUR CODE HERE
    Q=np.eye(n)
    if b==0:
        c=sign(a)
        s=0
    elif a==0:
        c=0
        s=-sign(b)
    else:  
        if a>=b:
            t=b/a
            c=sign(a)/np.sqrt(1+t**2)
            s=-c*t
        else:
            t=a/b
            s=-sign(b)/np.sqrt(1+t**2)
            c=-s*t
    
    Q[i,i]=c
    Q[j,j]=c
    Q[i,j]=-s
    Q[j,i]=s
    return(Q)

In [27]:
def givens_qr(A):
    """
    Computes a QR factorization of a matrix A by means of Givens
    rotations, where Q is an orthogonal matrix and R is upper-triangular. 
    
    Parameters
    ----------
        A (ndarray): A two-dimensional array of size n by m with n >= m
            whose columns are linearly independent
    
    Returns
    -------
        Q (ndarray): A two-dimensional array of size n by n
        R (ndarray): A two-dimensional array of size n by m
    """
    
    # YOUR CODE HERE
    n, m = A.shape
    Q = np.eye(n)
    R = A.copy()
    
    for j in range(m):
        for i in range(n-1, j, -1):
            if R[i, j] != 0:
                a = R[i-1, j]
                b = R[i, j]
                G = givens_rotation(i-1, i, a, b, n)
                R = G @ R
                Q = Q @ np.transpose(G)
    
    return Q, R
    

In [28]:
# You can use this code cell to play around with your functions to make sure
# they do what they are intended to do, i.e. to debug your code. 


In [29]:
# Test case for givens_rotation
np.set_printoptions(suppress=True) # Prints nicer numbers, you may ignore this line

Q_1 = givens_rotation(1, 2, 3, -4, 3)
print(Q_1, "\n")

Q_2 = givens_rotation(1, 2, 5, 12, 4)
print(Q_2, "\n")

Q_3 = givens_rotation(1, 3, -4, 0, 5)
print(Q_3)

[[ 1.   0.   0. ]
 [ 0.   0.6 -0.8]
 [ 0.   0.8  0.6]] 

[[ 1.          0.          0.          0.        ]
 [ 0.          0.38461538  0.92307692  0.        ]
 [ 0.         -0.92307692  0.38461538  0.        ]
 [ 0.          0.          0.          1.        ]] 

[[ 1.  0.  0.  0.  0.]
 [ 0. -1.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.]
 [ 0.  0.  0. -1.  0.]
 [ 0.  0.  0.  0.  1.]]


Expected output:

    [[ 1.   0.   0. ]
     [ 0.   0.6 -0.8]
     [ 0.   0.8  0.6]]

    [[ 1.          0.          0.          0.        ]
     [ 0.          0.38461538  0.92307692  0.        ]
     [ 0.         -0.92307692  0.38461538  0.        ]
     [ 0.          0.          0.          1.        ]]

    [[ 1.  0.  0.  0.  0.]
     [ 0. -1.  0.  0.  0.]
     [ 0.  0.  1.  0.  0.]
     [ 0.  0.  0. -1.  0.]
     [ 0.  0.  0.  0.  1.]]

In [30]:
# Test case I for givens_qr
A = np.array([[2,2], [1,7], [-2, -8]])
Q, R = givens_qr(A)
print(Q, "\n")
print(R)

[[ 0.66666667 -0.66666667  0.33333333]
 [ 0.33333333  0.66666667  0.66666667]
 [-0.66666667 -0.33333333  0.66666667]] 

[[ 3.  9.]
 [-0.  6.]
 [ 0. -0.]]


Expected output:

    [[ 0.66666667 -0.66666667  0.33333333]
     [ 0.33333333  0.66666667  0.66666667]
     [-0.66666667 -0.33333333  0.66666667]]
     
    [[3. 9.]
     [0. 6.]
     [0. 0.]]

In [31]:
# Test case II
B = np.array([[-1, 4, -1], [1, -1, -1], [1, -1, 1], [-1, 4, -3]])
Q, R = givens_qr(B)
print(Q, "\n")
print(R)

[[-0.5  0.5  0.5  0.5]
 [ 0.5  0.5 -0.5  0.5]
 [ 0.5  0.5  0.5 -0.5]
 [-0.5  0.5 -0.5 -0.5]] 

[[ 2. -5.  2.]
 [ 0.  3. -2.]
 [-0. -0.  2.]
 [ 0. -0.  0.]]


Expected output:

    [[-0.5  0.5  0.5  0.5]
     [ 0.5  0.5 -0.5  0.5]
     [ 0.5  0.5  0.5 -0.5]
     [-0.5  0.5 -0.5 -0.5]] 

    [[ 2. -5.  2.]
     [-0.  3. -2.]
     [-0.  0.  2.]
     [-0.  0.  0.]]

In [32]:
# AUTOGRADER for givens_rotation
Q_1 = givens_rotation(1, 2, 3, -4, 3)
assert np.allclose(Q_1, 1/5 * np.array([[5, 0, 0], [0, 3, -4], [0, 4, 3]]))

Q_2 = givens_rotation(1, 2, 5, 12, 4)
assert np.allclose(Q_2, 1/13 * np.array([[13, 0, 0, 0], [0, 5, 12, 0], [0,-12, 5, 0], [0, 0, 0, 13]]))

Q_3 = givens_rotation(1, 3, -4, 0, 5)
assert np.allclose(Q_3, np.array([[1, 0, 0, 0, 0], [0, -1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, -1, 0],
                                        [0, 0, 0, 0, 1]]))

In [33]:
# AUTOGRADER for givens_qr
A = np.array([[2,2], [1,7], [-2, -8]])
Q, R = givens_qr(A)
Q_exact, R_exact = 1/3*np.array([[2, -2, 1], [1, 2, 2], [-2, -1, 2]]), np.array([[3, 9], [0, 6], [0, 0]])

assert np.allclose(Q, Q_exact)
assert np.allclose(R, R_exact)


## Optional: Modified Gram-Schmidt Algorithm and Triangular Orthogonalization
### Feel free to skip.

The classical Gram-Schmidt algorithm is numerically unstable, which means that when implemented on a computer, round-off errors can cause the output vectors to be significantly non-orthogonal. This instability can be improved with a small adjustment to the algorithm. In the literature this remedy is called the modified Gram-Schmidt method. Instead of doing
$$v_k^\perp = v_k - \langle v_k, u_1 \rangle u_1 - \dots - \langle v_k, u_{k-1}\rangle u_{k-1}$$
we do the orthogonalization step-by-step. The modified algorithm calculates $u_j$ by evaluating the following formulas in order:
\begin{align*}
v_j^{(1)} &= v_j \\
v_j^{(2)} &= v_j^{(1)} - \langle v_j^{(1)}, u_1 \rangle u_1 \\
v_j^{(3)} &= v_j^{(2)} - \langle v_j^{(2)}, u_2 \rangle u_2 \\
v_j^{(4)} &= v_j^{(3)} - \langle v_j^{(3)}, u_3 \rangle u_3 \\
& ~~ \vdots \\
u_j = v_j^{(j)} &= v_j^{(j-1)} - \langle v_j^{(j-1)}, u_{j-1} \rangle u_{j-1}
\end{align*}

Once a new $u_i$ is known, we can overwrite each $v_j^{(i)}$ to $v_j^{(i+1)}$ for each $j>i$. Each step of the modified Gram-Schmidt algorithm can be interpreted as a right-multiplication by a square upper-triangular matrix. For example, beginning with $A$, the first iteration multiplies the first column $v_1$ by $\frac{1}{r_{11}} = \frac{1}{\|v_1\|}$ and then subtracts $r_{1j} = \langle v_j^{(1)}, u_1 \rangle = \frac{\langle v_j, v_1 \rangle}{\|v_1\|}$ times the result from each of the remaining columns $v_j$. This is equivalent to right-multiplication by a matrix $R_1$:

$$AR_1 = \begin{bmatrix}\mid & \mid & & \mid \\ v_1 & v_2 & \dots & v_m \\ \mid & \mid & & \mid \end{bmatrix} \begin{bmatrix} \frac{1}{r_{11}} & \frac{-r_{12}}{r_{11}} & \frac{-r_{13}}{r_{11}} & \dots \\ & 1 &  & \\ & & 1 &  \\ & & & \ddots \end{bmatrix} = \begin{bmatrix}\mid & \mid & & \mid \\ u_1 & v_2^{(2)} & \dots & v_m^{(2)} \\ \mid & \mid & & \mid \end{bmatrix}.$$

Empty entries signify zeroes. In general, in step $i$ the algorithm subtracts $r_{ij}/r_{ii}$ times column $i$ of the current $A$ from columns $j > i$ and replaces column $i$ by $1/r_{ii}$ times itself. This corresponds to multiplication by an upper-triangular matrix $R_i$:
$$R_2 = \begin{bmatrix} 1 & & & \\ & \frac{1}{r_{22}} & \frac{-r_{23}}{r_{22}} & \dots\\ & & 1 &  \\ & & & \ddots \end{bmatrix}, \quad R_3 = \begin{bmatrix} 1 & & & \\ & 1 & & \\ & & \frac{1}{r_{33}} & \dots \\ & & & \ddots \end{bmatrix}, \quad \dots$$
Recall that
\begin{align*}
r_{ii} &= \| v_i^{(i)} \|,  &&\text{for } 1 \leq i \leq m;\\
r_{ij} &= \langle u_i, v_j^{(i)} \rangle = \frac{\langle v_i^{(i)}, v_j^{(i)}\rangle}{\|v_i^{(i)}\|},  &&\text{for } 1 \leq i < j \leq m.
\end{align*}

After $i$ multiplications we have
$$A R_1 R_2 \dots R_i = \begin{bmatrix}\mid & \mid & & \mid & \mid & & \mid \\ u_1 & u_2 & \dots & u_i & v_{i+1}^{(i+1)} & \dots & v_{m}^{(i+1)} \\ \mid & \mid & & \mid & \mid & & \mid \end{bmatrix}$$

At the end of the iteration we have
$$A R_1 R_2 \dots R_n = Q,$$
and equivalently $A = QR = Q (R_1 R_2 \dots R_n)^{-1}.$ Hence the name triangular orthogonalization. 